# Face Detection

In [15]:
!pip install facenet-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [17]:
from facenet_pytorch import MTCNN, InceptionResnetV1

In [ ]:
help(MTCNN)

In [19]:
# If required, create a face detection pipeline using MTCNN:
mtcnn = MTCNN()

# Create an inception resnet (in eval mode):
resnet = InceptionResnetV1(pretrained='vggface2').eval()

  0%|          | 0.00/107M [00:00<?, ?B/s]

In [21]:
from PIL import Image

img = Image.open("group_photo.jpg")

# Get cropped and prewhitened image tensor
img_cropped = mtcnn(img, save_path="group_photo_saved.jpg")

# Calculate embedding (unsqueeze to add batch dimension)
img_embedding = resnet(img_cropped.unsqueeze(0))

# Or, if using for VGGFace2 classification
resnet.classify = True
img_probs = resnet(img_cropped.unsqueeze(0))

# Custom face comparison using pretrained resnet18

In [1]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.functional as F
import torchvision.models as models

In [5]:
class FaceNet(torch.nn.Module):
    def __init__(self):
        super(FaceNet, self).__init__()
        # Define the architecture here (e.g., CNN layers followed by fully connected layers)
        self.cnn_layers = torch.nn.Sequential(
            *list(models.resnet18(pretrained=True).children())[:-2]  # Remove last 2 layers (avgpool and fc)
        )
        # Modify the fully connected layer to output embeddings of desired size
        self.fc = torch.nn.Linear(512*7*7, 128)  # Adjust input size

    def forward(self, x):
        # Forward pass through the CNN layers
        x = self.cnn_layers(x)
        # Flatten the output for the fully connected layer
        x = torch.flatten(x, 1)
        # Forward pass through the fully connected layer
        x = self.fc(x)
        # Apply L2 normalization to the embeddings
        x = F.normalize(x, p=2, dim=1)
        return x

In [3]:
# Step 2: Preprocess the images
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize the image to fit the input size of the model
        transforms.ToTensor(),           # Convert image to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize image
    ])
    image = transform(image)
    return image.unsqueeze(0)  # Add batch dimension

# Step 3: Generate embeddings for the images
def generate_embedding(image_path, model):
    image_tensor = preprocess_image(image_path)
    with torch.no_grad():
        embedding = model(image_tensor)
    return embedding

# Step 4: Calculate similarity between embeddings
def calculate_similarity(embedding1, embedding2):
    # Calculate cosine similarity between the embeddings
    similarity = F.cosine_similarity(embedding1, embedding2)
    return similarity.item()

In [14]:
# Load FaceNet model
face_net_model = FaceNet()

# Load images and generate embeddings
image1_path = "Shah_Rukh_Khan.115.jpg"
image2_path = "Shah_Rukh_Khan.131.jpg"
embedding1 = generate_embedding(image1_path, face_net_model)
embedding2 = generate_embedding(image2_path, face_net_model)

# Calculate similarity between embeddings
similarity_score = calculate_similarity(embedding1, embedding2)
print("Similarity score between the images:", similarity_score)

Similarity score between the images: 0.489027738571167
